In [3]:
import json
import pandas as pd
import requests

## Create an temporal json

In [5]:
with open("./processed/dpts_cods_mpios.json", encoding="utf-8") as json_data:
    data = json.load(json_data)

In [12]:
new_data = []
for dept in data:
    tmp = {
        "department": dept,
        "municipalities": {
            muni: None for muni in data[dept]["municipalities"]
        }
    }
    new_data.append(tmp)

In [14]:
with open("./processed/depts_mpios_tmp.json", "w", encoding="utf-8") as json_data:
    json.dump(new_data, json_data, indent=4)

## Fill with geolocation data

In [4]:
with open("./processed/dpts_cods_mpios.json") as json_data:
    data = json.load(json_data)

vulnrb = pd.read_csv("./processed/vulnrb.csv")
processes = pd.read_csv("./processed/Procesos_casas_de_justicia.csv", index_col=0)

cod_depts = []
cod_mpios = []
arr = processes[["DEPARTAMENTO", "MUNICIPIO"]].to_numpy()
for pair in arr:
    cod_depts.append(data[pair[0]]["code"])
    cod_mpios.append(data[pair[0]]["municipalities"][pair[1]])

processes["COD_DEPTS"] = cod_depts
processes["COD_MPIOS"] = cod_mpios

processes["key"] = [f'{dept:02}{mpio:03}' for dept, mpio in processes[["COD_DEPTS", "COD_MPIOS"]].to_numpy()]
vulnrb["key"] = [f'{dept:02}{mpio:03}' for dept, mpio in vulnrb[["COD_DPTO", "COD_MPIO"]].to_numpy()]

df = pd.merge(processes, vulnrb, on="key")

In [5]:
df.drop(columns=["COD_DEPTS", "COD_MPIOS", "key", "COD_DPTO", "COD_MPIO"], inplace=True)
df

,IdSolicitud,NomEntidad,NomCasaJusticia,FechaSolicitud,DEPARTAMENTO,MUNICIPIO,days,months,years,day_of_week,VULNERABILITY,IPM-mean,IPM-median,PREGANCY,REACTIVATION
0,481137,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,CASA DE JUSTICIA CAJICA,2019-10-21,CUNDINAMARCA,CAJICA,21,10,2019,1,Vulnerabilidad baja,6.305179,4.511278,Vulnerabilidad baja,elegibilidad media-baja
1,3930874,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,CASA DE JUSTICIA CAJICA,2019-05-08,CUNDINAMARCA,CAJICA,8,5,2019,3,Vulnerabilidad baja,6.305179,4.511278,Vulnerabilidad baja,elegibilidad media-baja
2,414466,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,CASA DE JUSTICIA CAJICA,2019-09-16,CUNDINAMARCA,CAJICA,16,9,2019,1,Vulnerabilidad baja,6.305179,4.511278,Vulnerabilidad baja,elegibilidad media-baja
3,441614,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,CASA DE JUSTICIA CAJICA,2019-09-30,CUNDINAMARCA,CAJICA,30,9,2019,1,Vulnerabilidad baja,6.305179,4.511278,Vulnerabilidad baja,elegibilidad media-baja
4,39309,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,CASA DE JUSTICIA CAJICA,2019-05-08,CUNDINAMARCA,CAJICA,8,5,2019,3,Vulnerabilidad baja,6.305179,4.511278,Vulnerabilidad baja,elegibilidad media-baja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3607567,6998858,Conciliación en Equidad,CASA DE JUSTICIA SANTA MARTA,2024-09-30,MAGDALENA,SANTA MARTA,30,9,2024,1,Vulnerabilidad baja,20.374544,14.705882,Vulnerabilidad baja,elegibilidad media
3607568,6998861,Conciliación en Equidad,CASA DE JUSTICIA SANTA MARTA,2024-09-30,MAGDALENA,SANTA MARTA,30,9,2024,1,Vulnerabilidad baja,20.374544,14.705882,Vulnerabilidad baja,elegibilidad media
3607569,6998863,Conciliación en Equidad,CASA DE JUSTICIA SANTA MARTA,2024-09-30,MAGDALENA,SANTA MARTA,30,9,2024,1,Vulnerabilidad baja,20.374544,14.705882,Vulnerabilidad baja,elegibilidad media
3607570,6998867,Conciliación en Equidad,CASA DE JUSTICIA SANTA MARTA,2024-09-30,MAGDALENA,SANTA MARTA,30,9,2024,1,Vulnerabilidad baja,20.374544,14.705882,Vulnerabilidad baja,elegibilidad media


In [6]:
def get_coordinates(place_name: str) -> dict[str, str] | None:
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': str(place_name),      
        'format': 'json',      
        'addressdetails': 1,   
        'limit': 1             
    }

    headers = {
        'User-Agent': 'TuApp/1.0 (tucorreo@dominio.com)'
    }

    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        return None

    data = response.json()
    if not data:
        return None
    
    if data[0]['address']['country_code'] != 'co':
        return None

    lat = data[0]['lat']
    lon = data[0]['lon']
    return {"courthouse": place_name, "lat": lat, "lon": lon}


In [7]:
new_json = []
depts = df["DEPARTAMENTO"].unique()
for dept in depts:
    municipalities = {}
    sub_df = df.loc[df["DEPARTAMENTO"] == dept]
    mpios = sub_df["MUNICIPIO"].unique()
    for mpio in mpios:
        sub_df = sub_df.loc[sub_df["MUNICIPIO"] == mpio]
        courthouses = sub_df["NomCasaJusticia"].unique()
        if len(courthouses) < 1:
            municipalities[mpio] = None
            continue
        
        municipalities[mpio] = [
            r for q in courthouses if (r:=get_coordinates(q)) is not None
        ]
        
        if not any(municipalities[mpio]):
            municipalities[mpio] = None
    
    new_json.append({
        "department": dept,
        "municipalities": municipalities
    })


In [8]:
with open("./processed/depts_mpios.json", "w", encoding="utf-8") as json_data:
    json.dump(new_json, json_data, indent=4)

In [26]:
a = []
not any(a)

True